<a href="https://colab.research.google.com/github/Tstrebe2/predicting-text-difficulty/blob/tim-updates/code/tim-text-cleaning-pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

!{sys.executable} -m pip install pyspark==3.1.2 -q
!{sys.executable} -m pip install spark-nlp==4.2.0 -q

In [15]:
from google.colab import drive
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.ml import Pipeline
import pyspark.sql.functions as F
from sparknlp.base import DocumentAssembler, EmbeddingsFinisher
from sparknlp.annotator import SentenceDetector, Tokenizer, Normalizer, Lemmatizer
import sparknlp
from pyspark.sql.types import StringType, ArrayType, FloatType, StructType
import numpy as np
import string
import pandas as pd

In [3]:
# from google.colab import drive
drive.mount('/content/drive')

!wget https://raw.githubusercontent.com/Tstrebe2/predicting-text-difficulty/main/assets/WikiLarge_Train.csv -q
!wget https://raw.githubusercontent.com/mahavivo/vocabulary/master/lemmas/AntBNC_lemmas_ver_001.txt -q
!wget https://raw.githubusercontent.com/Tstrebe2/predicting-text-difficulty/main/assets/dale_chall.txt -q
!wget https://raw.githubusercontent.com/Tstrebe2/predicting-text-difficulty/main/assets/AoA_51715_words.csv -q
!wget https://raw.githubusercontent.com/Tstrebe2/predicting-text-difficulty/main/assets/Concreteness_ratings_Brysbaert_et_al_BRM.txt -q
# !wget http://nlp.stanford.edu/data/glove.6B.zip -q
# !unzip glove*.zip

Mounted at /content/drive


In [ ]:
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:4.2.0")\
        .getOrCreate()

## Readability Datasets (Provided by MADS)

In [3]:
def get_readability_datasets():
  aoa = pd.read_csv('/content/AoA_51715_words.csv', 
                      encoding_errors='ignore', 
                      usecols=['Lemma_highest_PoS', 'AoA_Kup_lem'],
                      ).rename({'Lemma_highest_PoS':'lemma', 'AoA_Kup_lem':'aoa'}, axis=1)

  aoa = aoa.groupby('lemma').first().to_dict()['aoa']

  conc = (pd.read_csv('/content/Concreteness_ratings_Brysbaert_et_al_BRM.txt', 
                    sep='\t',
                    usecols=['Word', 'Bigram', 'Conc.M'])
          .rename({'Word':'word', 'Bigram':'bigram', 'Conc.M':'conc_mean'}, axis=1))

  def split_word(x):
    if x['bigram'] == 0:
      word_or_phrase = x['word']
    else:
      word_or_phrase = tuple(x['word'].split(' '))

    return {'word':word_or_phrase, 'conc_mean':x['conc_mean'] }

  conc = conc.apply(split_word, axis=1, result_type='expand').set_index('word').to_dict()['conc_mean']

  d_chall = set(pd.read_csv('/content/dale_chall.txt', names=['word'])['word'].tolist())
  return aoa, conc, d_chall

## Core Datasets
* Standars unit-conversion metrics
* Clean punctuation
* Lower-case
* Lemmatize
* Create readability feature representations using MADS provided datasets


In [37]:
def get_clean_data_frame(data_path):
  aoa, conc, d_chall = get_readability_datasets()

  df = spark.read.csv(data_path, header=True)
  df.createOrReplaceTempView('wiki')

  regex1, replace1 = r" km ", "kilometers"
  regex2, replace2 = r"[0-9]+(km) ", "kilometers"
  regex3, replace3 = r" mph "," miles per hour "
  regex4, replace4 = r"° C ","degrees celsius"
  regex5, replace5 = r"° F ","degrees farenheit"
  regex6, replace6 = r"°","degrees"
  regex7, replace7 = r" %"," percent"
  regex8, replace8 = r" cm"," centimeters"
  regex9, replace9 = r" kg "," kilograms "

  iterable = ((regex1, replace1), (regex2, replace2), (regex3, replace3),
              (regex4, replace4), (regex5, replace5), (regex6, replace6),
              (regex7, replace7), (regex8, replace8), (regex9, replace9),)

  for regex, replace in iterable:
    query = f"""
    SELECT
      regexp_replace(original_text, '{regex}', '{replace}') as original_text, 
      label 
    FROM wiki;"""
    df = spark.sql(query)
    df.createOrReplaceTempView('wiki')

  documentAssembler = DocumentAssembler()\
      .setInputCol("original_text")\
      .setOutputCol("document")

  tokenizer = Tokenizer() \
      .setInputCols(["document"]) \
      .setOutputCol("token")

  normalizer = Normalizer() \
      .setInputCols(["token"]) \
      .setOutputCol("normalized") \
      .setLowercase(True) \
      .setCleanupPatterns(["""[^\w\d\s]"""]) # remove punctuations (keep alphanumeric chars)
  # if we don't set CleanupPatterns, it will only keep alphabet letters ([^A-Za-z])

  lemmatizer = Lemmatizer() \
      .setInputCols(["normalized"]) \
      .setOutputCol("lemma") \
      .setDictionary("./AntBNC_lemmas_ver_001.txt", value_delimiter ="\t", key_delimiter = "->")

  nlp_pipeline = Pipeline(stages=[documentAssembler,
                                  tokenizer,
                                  normalizer,
                                  lemmatizer,])

  nlp_pipeline = nlp_pipeline.fit(df)
  df = nlp_pipeline.transform(df)
  df.createOrReplaceTempView('wiki')

  def get_d_chall(x):
    easy_count = 0
    word_count = 0

    for token in x:
      if token in string.punctuation:
        continue

      token_lower = token.lower()

      if token_lower in d_chall:
        easy_count += 1

      word_count += 1

    difficult_count = (word_count-easy_count)

    return (0.1579 * ((difficult_count/word_count) * 100) + 0.0496 * word_count
            if word_count else 0.0)


  def get_aoa(x):
    arr = [aoa[w.lower()] for w in x if w.lower() in aoa]
    if len(arr) > 0:
      return arr
    else:
      return [0.0] 

  def get_conc_rating(x):
    ret_val = []

    bigrams = [(f.lower(), s.lower()) for f, s in zip(x[:-1], x[1:])]
    cont = False
    
    for bigram in bigrams:
      if cont:
        cont = False
        continue

      if bigram in conc:
        cont = True
        ret_val.append(conc[bigram])
      elif bigram[0] in conc:
        ret_val.append(conc[bigram[0]])

    return ret_val

  def get_num_lemmas(x):
    num_lemmas = 0.0

    for token in x:
      if token not in string.punctuation:
        num_lemmas += 1.0

    return num_lemmas
      
  spark.udf.register('get_d_chall', get_d_chall, FloatType())
  spark.udf.register('get_aoa', get_aoa, ArrayType(FloatType()))
  spark.udf.register('get_conc_rating', get_conc_rating, ArrayType(FloatType()))
  spark.udf.register('get_num_lemmas', get_num_lemmas, FloatType())
  spark.udf.register('get_joined_text', lambda x: ' '.join(x), StringType())
  spark.udf.register('array_mean', lambda x: float(np.mean(x)), FloatType())

  query = r"""
  SELECT 
    original_text, 
    get_joined_text(lemma.result) as lemmatized_text, 
    get_d_chall(lemma.result) as d_chall_score,
    get_aoa(lemma.result) as aoa,
    get_conc_rating(lemma.result) as conc_rating,
    get_num_lemmas(lemma.result) as num_lemmas,
    label
  FROM wiki;
  """
  df = spark.sql(query)
  df.createOrReplaceTempView('wiki')

  spark.udf.register('array_mean', lambda x: float(np.mean(x)), FloatType())

  query = r"""
  SELECT 
  original_text, 
  lemmatized_text,
  d_chall_score,
  array_mean(aoa) as aoa_mean, 
  array_min(aoa) as aoa_min, 
  array_max(aoa) as aoa_max, 
  array_mean(conc_rating) as conc_rating_mean, 
  array_min(conc_rating) as conc_rating_min, 
  array_max(conc_rating) as conc_rating_max,
  num_lemmas,
  label
  FROM wiki; 
  """
  df = spark.sql(query)
  df.createOrReplaceTempView('wiki')
  return df.toPandas()

In [38]:
df = get_clean_data_frame(data_path='/content/WikiLarge_Train.csv')
df.shape

(416768, 11)

In [39]:
def save_train_test_split(df, train_path, test_path):
  df_test = df.groupby('label').apply(lambda grp: grp.sample(frac=.10, random_state=99)).droplevel(0, axis=0)
  df_train = df[~df.index.isin(df_test.index)]

  df_train.to_csv(train_path, sep='\t', index=True, index_label='ix')
  df_test.to_csv(test_path, sep='\t', index=True, index_label='ix')

save_train_test_split(df, 
                      train_path='/content/drive/MyDrive/milestone-ii/Training_set.csv',
                      test_path='/content/drive/MyDrive/milestone-ii/Testing_set.csv')

In [44]:
df[df.lemmatized_text.str.len() == 0]

,original_text,lemmatized_text,d_chall_score,aoa_mean,aoa_min,aoa_max,conc_rating_mean,conc_rating_min,conc_rating_max,num_lemmas,label
607,! '',,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,1
2118,.,,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,1
3821,.,,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,1
5949,!,,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,1
6570,-,,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...
409118,.,,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0
411367,.,,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0
414899,.,,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0
414941,!,,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0


## Word Embeddings

In [ ]:
# https://edumunozsala.github.io/BlogEms/jupyter/nlp/classification/embeddings/python/2020/08/15/Intro_NLP_WordEmbeddings_Classification.html
# https://radimrehurek.com/gensim_3.8.3/models/keyedvectors.html
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

root_path = '/content/'
glove_file_name = 'glove.6B.100d.txt'
glove_path = ''.join([root_path, glove_file_name])

word2vec_output_file = ''.join([glove_file_name, '.word2vec'])
glove2word2vec(glove_path, word2vec_output_file)

model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

def get_sentence_embedding(sent):
  sentence_embedding = np.zeros((0, 100))
  tokens = sent.split(' ')
  
  for token in tokens:
    token_lower = token.lower()

    if token_lower in model:
      word_vector = model.get_vector(token_lower)
      sentence_embedding = np.vstack((sentence_embedding, word_vector))

  if sentence_embedding.shape[0] == 0:
    return np.zeros(100)
  else:
    return sentence_embedding.mean(axis=0)

In [ ]:
embeddings = np.array(df['original_text'].apply(get_sentence_embedding).tolist())
embeddings.shape

(395169, 100)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

y = df['label'].astype(np.int64)
X_train, X_test, y_train, y_test = train_test_split(embeddings, y, test_size=.1, random_state=99)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((355652, 100), (39517, 100), (355652,), (39517,))

In [ ]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler

clf = make_pipeline(StandardScaler(), LogisticRegression(n_jobs=-1)).fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
y_hat = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)[:, 1]

roc_auc = roc_auc_score(y_test, y_proba)
accuracy = accuracy_score(y_test, y_hat)
f1 = f1_score(y_test, y_hat)

print('ROC AUC:', roc_auc)
print('Accuracy:', accuracy)
print('F1:', f1)

ROC AUC: 0.6662212002745456
Accuracy: 0.6354986461522889
F1: 0.6687669594812123


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df[['lemmatized_text']], y, test_size=.1, random_state=99)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((355652, 1), (39517, 1), (355652,), (39517,))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

clf = make_pipeline(TfidfVectorizer(min_df=25, stop_words='english'), 
                    LogisticRegression(n_jobs=-1)).fit(X_train['lemmatized_text'], y_train)

In [ ]:
y_hat = clf.predict(X_test['lemmatized_text'])
y_proba = clf.predict_proba(X_test['lemmatized_text'])[:, 1]

roc_auc = roc_auc_score(y_test, y_proba)
accuracy = accuracy_score(y_test, y_hat)
f1 = f1_score(y_test, y_hat)

print('ROC AUC:', roc_auc)
print('Accuracy:', accuracy)
print('F1:', f1)

ROC AUC: 0.730130701783159
Accuracy: 0.6654098236202141
F1: 0.6759631408685423
